# Load GAVEL games

In [7]:
import pandas as pd

game_feature_csv_paths = [
    'generated_csvs/fold_0_games_and_features.csv',
    'generated_csvs/fold_1_games_and_features.csv',
    'generated_csvs/fold_2_games_and_features.csv',
    'generated_csvs/fold_3_games_and_features.csv',
    'generated_csvs/fold_4_games_and_features.csv',
    'generated_csvs/fold_5_games_and_features.csv',
]

# Parallel to csv paths. 9950x = 1.82, 5800x3d = 1.51, 5950x = 0.96.
# Used to normalize playouts per second & moves per second to be similar to organizer.
annotation_machine_throughput_coefs = [
    1.82,
    1.82,
    0.96,
    1.82,
    1.82,
    0.96,
]

game_features_dfs = []
for csv_path, machine_throughput_coef in zip(game_feature_csv_paths, annotation_machine_throughput_coefs):
    game_features_df = pd.read_csv(csv_path)
    game_features_df['PlayoutsPerSecond'] /= machine_throughput_coef
    game_features_df['MovesPerSecond'] /= machine_throughput_coef

    game_features_dfs.append(game_features_df)

game_features_df = pd.concat(game_features_dfs)

In [2]:
import hashlib
import json

import sys
sys.path.append('../DataGeneration/NewGameGeneration')
from RuleMinification import MinifyRules

elite_lud_filepaths = [
    'elite_generated_luds/fold_0.json',
    'elite_generated_luds/fold_1.json',
    'elite_generated_luds/fold_2.json',
    'elite_generated_luds/fold_3.json',
    'elite_generated_luds/fold_4.json',
    'elite_generated_luds/fold_5.json',
]
all_elite_luds = []
for elite_lud_filepath in elite_lud_filepaths:
    with open(elite_lud_filepath, 'r') as f:
        all_elite_luds.extend(json.load(f))


individual_minified_luds_to_hashes = {}
for lud in all_elite_luds:
    minified_lud = MinifyRules(lud)
    lud_hash = hashlib.sha256(lud.encode()).hexdigest()
    individual_minified_luds_to_hashes[minified_lud] = lud_hash

In [3]:
import glob
import os
import re
import numpy as np

minified_luds_to_agents_to_scores: dict[str: dict[tuple[str, str]: float]] = {}

for minified_lud, lud_hash in individual_minified_luds_to_hashes.items():
    minified_luds_to_agents_to_scores[minified_lud] = {}

    annotation_config_filepaths = glob.glob(f'configs/match_configs/*/{lud_hash[:8]}_*.json')
    agent_strings = set()
    for annotation_config_filepath in annotation_config_filepaths:
        with open(annotation_config_filepath, 'r') as f:
            annotation_config = json.load(f)
            agent_pair = annotation_config['agentStrings']
        
        agent_pair_hash = hashlib.sha256(json.dumps(agent_pair).encode()).hexdigest()[:8]

        match_results_filepath = f'ludii_player_outputs/matches/game_{lud_hash[:8]}/match_{agent_pair_hash}/raw_results.csv'
        if not os.path.exists(match_results_filepath):
            continue

        raw_results_df = pd.read_csv(match_results_filepath)
        
        readable_agent_pairs_to_outcomes = {}
        for _, row in raw_results_df.iterrows():
            agent_pair_text = row['agents']
            
            agent_pattern = r"\('(.*?)' \/ '(.*?)'\)"
            agent_matches = re.match(agent_pattern, agent_pair_text)
            assert (agent_matches is not None) and (len(agent_matches.groups()) == 2)

            readable_agent_pair = (agent_matches.group(1), agent_matches.group(2))

            outcome = float(row['utilities'].split(';')[0])

            if readable_agent_pair not in readable_agent_pairs_to_outcomes:
                readable_agent_pairs_to_outcomes[readable_agent_pair] = []

            readable_agent_pairs_to_outcomes[readable_agent_pair].append(outcome)

        for agent_pair, outcomes in readable_agent_pairs_to_outcomes.items():
            assert len(outcomes) == 4 or (agent_pair[0] == agent_pair[1] and len(outcomes) == 8)
            minified_luds_to_agents_to_scores[minified_lud][agent_pair] = np.mean(outcomes)


In [4]:
print(len([
    lud
    for lud, agent_pairs_to_scores in minified_luds_to_agents_to_scores.items() 
    if len(agent_pairs_to_scores) > 0
]))

print([
    len(agent_pairs_to_scores)
    for lud, agent_pairs_to_scores in minified_luds_to_agents_to_scores.items() 
    if len(agent_pairs_to_scores) > 0
])

368
[30, 30, 29, 28, 28, 30, 30, 30, 29, 30, 29, 30, 30, 30, 30, 30, 30, 29, 30, 29, 30, 28, 30, 30, 30, 30, 30, 29, 30, 29, 30, 29, 29, 30, 30, 30, 30, 29, 30, 30, 29, 30, 30, 30, 30, 28, 28, 30, 30, 29, 30, 30, 30, 30, 30, 30, 29, 29, 30, 29, 30, 30, 30, 30, 29, 30, 30, 30, 29, 30, 29, 29, 28, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 28, 30, 30, 30, 29, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 29, 30, 30, 30, 30, 29, 30, 29, 29, 28, 30, 30, 28, 30, 30, 30, 29, 30, 29, 30, 30, 30, 30, 30, 30, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 28, 30, 30, 30, 30, 29, 26, 30, 29, 30, 29, 28, 29, 29, 30, 30, 30, 28, 30, 29, 30, 30, 28, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 30, 30, 29, 28, 30, 30, 30, 30, 29, 30, 30, 30, 30, 30, 29, 30, 29, 30, 30, 27, 29, 30, 30, 30, 29, 30, 30, 30, 30, 30, 30, 30, 30, 29, 29, 29, 28, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 29, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 30, 29, 30, 30,

In [8]:
column_names_to_values = {
    column_name: []
    for column_name in game_features_df.columns
}
column_names_to_values['utility_agent1'] = []
column_names_to_values['agent1'] = []
column_names_to_values['agent2'] = []

missing_lud_count = 0
extraneous_lud_count = 0
for lud, agents_to_scores in minified_luds_to_agents_to_scores.items():
    # LOAD GAME FEATURES
    game_features_row = game_features_df[game_features_df['LudRules'] == lud]
    if len(game_features_row) == 0:
        missing_lud_count += 1
        print(f'Missing features for lud {lud}')
        continue
    if len(game_features_row) > 1:
        extraneous_lud_count += 1
        print(f'Found multiple rows for lud {lud}')
        continue

    assert len(game_features_row) == 1
    game_features_row = game_features_row.iloc[0]

    # ADD ROW FOR EACH EVALUATED AGENT PAIR.
    for (agent1, agent2), score in agents_to_scores.items():
        for column_name in game_features_df.columns:
            column_names_to_values[column_name].append(game_features_row[column_name])

        column_names_to_values['utility_agent1'].append(score)
        column_names_to_values['agent1'].append(agent1)
        column_names_to_values['agent2'].append(agent2)

evaluated_games_df = pd.DataFrame(column_names_to_values)

print(f'Missing features for {missing_lud_count} luds')
print(f'Extraneous features for {extraneous_lud_count} luds')

Found multiple rows for lud (game "Fart (7x7) " (players 2) (equipment { (board (square 7)) (piece "Marker" Each (move Slide Orthogonal)) (hand Each) } ) (rules (start (place "Marker" "Hand" count:24)) phases:{ (phase "Placement" (play (move (from (handSite Mover)) (to (difference (sites Board) (centrePoint)) if:(is Empty (to)) ) (then (if (not (is Prev Mover)) (moveAgain))) ) ) (nextPhase Mover (is Empty (handSite Mover)) "Move") ) (phase "Move" (play (forEach Piece))) } ) )
Found multiple rows for lud (game "Boxijn Barildaan" (players 2) (equipment { (board (merge { (shift 1 0 (rectangle 4 2)) (shift 0 1 (rectangle 2 4)) } ) use:Vertex ) (piece "Marker" Each (move Step (to if: (and (= 0 (state at:(to))) (if (< (count MovesThisTurn) 2) (is Empty (to)) (is In (to) (union (sites Occupied by:Next) (sites Empty) ) ) ) ) (apply (if (is Enemy (who at:(to))) (remove (to)))) ) (then (if (= 2 (count Sites in:(forEach (sites Board) if:(= 1 (state at:(site))) ) ) ) (forEach Site (sites Board) (i

# Load non-GAVEL games

In [ ]:
# Names like {game_hash}/Concepts.csv
!ls -l ../DataGeneration/GameFeatures/SyntheticGames/Batch1/ | wc -l

# Names like game_{game_hash}/agent_pair_{agent_pair_hash}/raw_results.csv
!ls -l ../DataGeneration/Matches/Batch1/ | wc -l

# Names like {game_hash}.lud
!ls -l ../DataGeneration/NewGameGeneration/GeneratedGames_qwen_5shot_V2/ | wc -l

350
119
15791


In [ ]:
from tqdm import tqdm

feature_directory_paths = glob.glob('../DataGeneration/GameFeatures/SyntheticGames/Batch1/*')
for game_index, feature_directory_path in tqdm(enumerate(feature_directory_paths)):
    # # DOWNSAMPLE.
    # if game_index % 4 != 0:
    #     continue

    # VERIFY MATCH OUTCOMES EXIST.
    game_hash = os.path.basename(feature_directory_path)
    match_outcomes_directory_path = f'../DataGeneration/Matches/Batch1/game_{game_hash}'
    if not os.path.exists(match_outcomes_directory_path):
        continue

    # LOAD GAME FEATURES.
    game_features_df = pd.read_csv(f'{feature_directory_path}/Concepts.csv')

    # LOAD RULESET NAME.
    with open(f'../DataGeneration/NewGameGeneration/GeneratedGames_qwen_5shot_V2/{game_hash}.json', 'r') as f:
        raw_game_data = json.load(f)
        ruleset_name = raw_game_data['game_name']
        lud_rules = raw_game_data['lud_rules']
    
    # LOAD MATCH OUTCOMES.
    match_outcomes_filepaths = glob.glob(f'{match_outcomes_directory_path}/*/raw_results.csv')
    for match_outcomes_filepath in match_outcomes_filepaths:
        with open(match_outcomes_filepath, 'r') as f:
            raw_results_df = pd.read_csv(f)
        
        agent_pairs_to_outcomes: dict[tuple[str, str]: list[float]] = {}
        for _, row in raw_results_df.iterrows():
            agent_pair_text = row['agents']
            agent_pattern = r"\('(.*?)' \/ '(.*?)'\)"
            agent_matches = re.match(agent_pattern, agent_pair_text)
            assert (agent_matches is not None) and (len(agent_matches.groups()) == 2)

            agent_pair = (agent_matches.group(1), agent_matches.group(2))

            if agent_pair not in agent_pairs_to_outcomes:
                agent_pairs_to_outcomes[agent_pair] = []

            outcome = float(row['utilities'].split(';')[0])
            agent_pairs_to_outcomes[agent_pair].append(outcome)

        for (agent1, agent2), outcomes in agent_pairs_to_outcomes.items():
            new_record = game_features_df.copy()

            existing_column_count = len(new_record.columns)
            new_record.insert(existing_column_count, 'utility_agent1', [np.mean(outcomes)])
            new_record.insert(existing_column_count + 1, 'agent1', [agent1])
            new_record.insert(existing_column_count + 2, 'agent2', [agent2])
            new_record.insert(existing_column_count + 3, 'GameRulesetName', [ruleset_name])
            new_record.insert(existing_column_count + 4, 'LudRules', [lud_rules])

            new_record['PlayoutsPerSecond'] /= 1.3
            new_record['MovesPerSecond'] /= 1.3

            evaluated_games_df = pd.concat([evaluated_games_df, new_record])

349it [01:17,  4.50it/s]


# Generate starting position evaluations

In [14]:
from lib.Game import Game
from concurrent.futures import ThreadPoolExecutor

RUNTIME_PER_GAME_SECONDS = 16
# SELECTION = 'UCB1Tuned'
# EXPLORATION_CONST = '1.41421356237'
# PLAYOUT = 'random'
# SCORE_BOUNDS = 'false'

SELECTION = 'UCB1Tuned'
EXPLORATION_CONST = '0.6'
PLAYOUT = 'NST'
SCORE_BOUNDS = 'true'

unique_luds = evaluated_games_df['LudRules'].unique()
evaluation_futures = []
with ThreadPoolExecutor(max_workers=3) as executor:
    for lud in unique_luds:
        game = Game(lud, path_prefix='../')
        result_future = executor.submit(
            game.GetStartingPositionEvaluation, 
            selection = SELECTION,
            exploration_constant = EXPLORATION_CONST,
            playout = PLAYOUT.lower(),
            score_bounds = SCORE_BOUNDS,
            analysis_runtime_seconds = RUNTIME_PER_GAME_SECONDS, 
            regenerate = True)
        evaluation_futures.append((lud, result_future))

luds_to_starting_position_evaluations = {}
for lud, future in evaluation_futures:
    luds_to_starting_position_evaluations[lud] = future.result()

output_directory_path = f'generated_csvs/complete_datasets/{len(unique_luds)}'
os.makedirs(output_directory_path, exist_ok=True)

runtime_text = f'{RUNTIME_PER_GAME_SECONDS}s'
if RUNTIME_PER_GAME_SECONDS < 1:
    runtime_text = f'{int(RUNTIME_PER_GAME_SECONDS * 1000)}ms'

config_text = f'MCTS-{SELECTION}-{EXPLORATION_CONST}-{PLAYOUT}-{SCORE_BOUNDS}-{runtime_text}'

output_filepath = f'{output_directory_path}/{config_text}.json'
with open(output_filepath, 'w') as f:
    json.dump(luds_to_starting_position_evaluations, f)

ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/81526f0b01fa58dfb6135350c6536a889274a70ac3d492095b6729ca90bbbf89.lud
Ruleset name = Standard
ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/533a00613402e21a515c91ebdcc1a17f2940416f0e8b842281a147252805d14f.lud
Ruleset name = Standard
ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/2b9494fe6d3108ef62603d116260ddbf946076f8ccda19d8f21f0eeace358c24.lud
Ruleset name = Standard


Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/81526f0b01fa58dfb6135350c6536a889274a70ac3d492095b6729ca90bbbf89.txt
Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/533a00613402e21a515c91ebdcc1a17f2940416f0e8b842281a147252805d14f.txt
Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/2b9494fe6d3108ef62603d116260ddbf946076f8ccda19d8f21f0eeace358c24.txt


ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/601b390399992d029ff148b5805b97066a8d8e7bdea881c30a33119b06bbcef5.lud
Ruleset name = Standard
ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/949430434058436500b95855984f02b51fb2d5670c026c7981fabe65b8177098.lud
Ruleset name = Standard
ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/48fbbe5a9b6af81fb3032905558516d87b97ebbe6bdc7b82dd73214c50fd037b.lud
Ruleset name = Standard


Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/601b390399992d029ff148b5805b97066a8d8e7bdea881c30a33119b06bbcef5.txt
Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/949430434058436500b95855984f02b51fb2d5670c026c7981fabe65b8177098.txt
Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/48fbbe5a9b6af81fb3032905558516d87b97ebbe6bdc7b82dd73214c50fd037b.txt


ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/cff40067965902a376a79769f5abee03b51b4bfe7eb278a1e5d4b449d679cc42.lud
Ruleset name = Standard
ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/21a5117a58ebd788881c170124e6fb38ac30e3082dd4d482b9e4ea77a24bd2cd.lud
Ruleset name = Standard
ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/114866fbdf364aef8600528b84935217bdef71903793af190b35e6105c0b5274.lud
Ruleset name = Standard


...

Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/877128f97d6e96f67f7e86b6d751f764ab00bdd4866b809190dbe6f6b2060319.txt


ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/e0a787fc2f83a9cde290e482dc72d10fab5db6317df56d3d336079fa7135270b.lud
Ruleset name = Standard


Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/3bd3fec164bc7f81b3c5e374d077e2ac63473f0da1f174d1496cb03877a40f21.txt


ERROR: Ruleset name not found, loading default game options
Game file = /media/shared/ExpansionDrive2/TreeSearch/GAVEL/../GAVEL/generated_luds/4b7649d69227737e4bb3c4ee8ae8239023c57db39302944a0a43ab02e47d1d5b.lud
Ruleset name = Standard


Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/b8176755088933069c4dde5e170451d2f55008617fcfa12eee0c5c16915945df.txt
Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/e0a787fc2f83a9cde290e482dc72d10fab5db6317df56d3d336079fa7135270b.txt
Evaluation written to: ../GAVEL/ludii_player_outputs/starting_position_evaluations/4b7649d69227737e4bb3c4ee8ae8239023c57db39302944a0a43ab02e47d1d5b.txt


In [9]:
print(len(luds_to_starting_position_evaluations))
print(len([v for k,v in luds_to_starting_position_evaluations.items() if v is not None]))

233
232


# Save data

In [11]:
import datetime

timestamp_text = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
evaluated_games_df.to_csv(f'generated_csvs/complete_datasets/{timestamp_text}.csv', index=False)

In [12]:
print(f'Saved {len(evaluated_games_df)} rows for {len(set(evaluated_games_df["LudRules"]))} unique games.')

Saved 14247 rows for 484 unique games.


# Generate filtered subset with only new rows

In [13]:
old_evaluated_games_df = pd.read_csv('generated_csvs/complete_datasets/2024-11-08_16-32-19.csv')

old_luds = set(old_evaluated_games_df['LudRules'])
new_rows_df = evaluated_games_df[~evaluated_games_df['LudRules'].isin(old_luds)]

new_rows_df.to_csv(f'generated_csvs/complete_datasets/{timestamp_text}_new.csv', index=False)

In [17]:
print(f'{len(new_rows_df)}/{len(evaluated_games_df)} rows are new')
print(f'{len(set(new_rows_df["LudRules"]))}/{len(set(evaluated_games_df["LudRules"]))} games are new')

4669/14247 rows are new
158/484 games are new


# Sanity check data

LightGBM should be able to predict the utility_agent1 column more accurately than blind guessing.

In [18]:
import polars as pl

sys.path.append('..')
from TrainLGBM import TrainModels
from ColumnNames import DROPPED_COLUMNS, AGENT_COLS

EXTRA_DROPPED_COLUMNS = ['SkillTraceTrials', 'SkillTraceScore', 'SkillTrace', 'SkillTraceErrorIntercept', 'SkillTraceErrorSlope']
DROPPED_COLUMNS += EXTRA_DROPPED_COLUMNS

ruleset_names = evaluated_games_df['GameRulesetName']

preprocessed_df = pl.DataFrame(evaluated_games_df)
preprocessed_df = preprocessed_df.drop(filter(lambda x: x in preprocessed_df.columns, DROPPED_COLUMNS))

for col in AGENT_COLS:
    preprocessed_df = preprocessed_df.with_columns(pl.col(col).str.split(by="-").list.to_struct(fields=lambda idx: f"{col}_{idx}")).unnest(col).drop(f"{col}_0")

preprocessed_df = preprocessed_df.with_columns([pl.col(col).cast(pl.Categorical) for col in preprocessed_df.columns if col[:6] in AGENT_COLS])            
preprocessed_df = preprocessed_df.with_columns([pl.col(col).cast(pl.Float32) for col in preprocessed_df.columns if col[:6] not in AGENT_COLS])

preprocessed_df = preprocessed_df.to_pandas()


/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: /home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.so)

In [13]:
allowed_columns = [
    # Game.
    'AdvantageP1', 
    'Balance',
    'PlayoutsPerSecond', 
    'MovesPerSecond',
    'DurationTurnsStdDev',
    # Label.
    'utility_agent1'
# Agents.
] + [col for col in preprocessed_df.columns if col[:6] in AGENT_COLS]

seed_rmses = []
for seed in range(10):
    models, mean_rmse = TrainModels(
        ruleset_names = ruleset_names, 
        train_test_df = preprocessed_df[allowed_columns], 
        lgb_params = {
            # "num_iterations": 10000,
            # "learning_rate": 0.03184567466358953,
            # "num_leaves": 196,
            # "max_depth": 17,
            # "min_child_samples": 94,
            # "subsample": 0.8854325308371437,
            # "colsample_bytree": 0.9612980174610098,
            # "colsample_bynode": 0.6867101420064379,
            # "reg_alpha": 1.593152807295967e-05,
            # "reg_lambda": 4.8636580199114866e-08,
            # "extra_trees": False
        }, 
        early_stopping_round_count = 50, 
        fold_count = 10,
        random_seed = seed
    )
    seed_rmses.append(mean_rmse)

print(f'LGBM RMSE (mean, std): {np.mean(seed_rmses):.4f}, {np.std(seed_rmses):.4f}')

Fold 1/10...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000114 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 640
[LightGBM] [Info] Number of data points in the train set: 5478, number of used features: 13
[LightGBM] [Info] Start training from score 0.055472
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[23]	valid_0's rmse: 0.540331	valid_0's l2: 0.291958
Fold 2/10...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 740
[LightGBM] [Info] Number of data points in the train set: 6425, number of used features: 13
[LightGBM] [Info] Start training from score 0.078327
Training until va

In [14]:
from sklearn.metrics import mean_squared_error

dummy_prediction = np.mean(evaluated_games_df['utility_agent1'])
dummy_predictions = [dummy_prediction] * len(evaluated_games_df)

rmse = mean_squared_error(evaluated_games_df['utility_agent1'], dummy_predictions, squared=False)

print(f'Dummy prediction: {dummy_prediction:.4f}')
print(f'Dummy RMSE: {rmse:.4f}')

Dummy prediction: 0.0837
Dummy RMSE: 0.6695


/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
improvement_z_scores = (rmse - np.mean(seed_rmses)) / np.std(seed_rmses)
print(f'Improvement Z-score: {improvement_z_scores:.4f}')

Improvement Z-score: 8.7309


In [16]:
import joblib

# pretrained_model = joblib.load('../models/catboost_41810_10_13iter_100round/0.p')
pretrained_model = joblib.load('../models/lgbm_42154_10_200iter_200round/0.p')

# expected_feature_names = set(pretrained_model.feature_names_)
# actual_feature_names = set(test_x.columns)
# print('Missing features:', expected_feature_names - actual_feature_names)
# print('Extra features:', actual_feature_names - expected_feature_names)

test_x = preprocessed_df.drop(['utility_agent1'], axis=1)
predictions = pretrained_model.predict(test_x)
rmse = mean_squared_error(evaluated_games_df['utility_agent1'], predictions, squared=False)

print(np.mean(predictions), np.mean(evaluated_games_df['utility_agent1']))

print(rmse)

0.2992366321674854 0.08366372980910426
0.7543462005222081


/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
from scipy.stats import pearsonr

pearsonr(evaluated_games_df['utility_agent1'], predictions)

PearsonRResult(statistic=-0.09466458347102, pvalue=4.951826858873004e-15)

In [18]:
print(pearsonr(evaluated_games_df['AdvantageP1'], predictions))
print(pearsonr(evaluated_games_df['AdvantageP1'], evaluated_games_df['utility_agent1']))

PearsonRResult(statistic=-0.03647729280573812, pvalue=0.0026068496133092645)
PearsonRResult(statistic=0.535934292771505, pvalue=0.0)


In [19]:
# evaluated_games_df[['LudRules', 'agent1', 'agent2', 'utility_agent1']].head(60)

In [20]:
from TrainLGBM import GetPreprocessedData

_, organizer_df = GetPreprocessedData(split_agent_features=True)

test_x = organizer_df[allowed_columns].drop(['utility_agent1'], axis=1)
# predictions = models[3].predict(test_x)

predictions = []
for model in models:
    predictions.append(model.predict(test_x))

# predictions = (1 - np.std(predictions, axis=0))*np.mean(predictions, axis=0)
predictions = np.mean(predictions, axis=0)

print(mean_squared_error(organizer_df['utility_agent1'], predictions, squared=False))
print(pearsonr(organizer_df['utility_agent1'], predictions))

print()
print(pearsonr(organizer_df['utility_agent1'], organizer_df['AdvantageP1']))
print(pearsonr(predictions, organizer_df['AdvantageP1']))

/media/shared/ExpansionDrive2/TreeSearch/GAVEL/../TrainLGBM.py:30: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = df.with_columns([pl.col(col).cast(pl.Categorical) for col in df.columns if col[:6] in AGENT_COLS])


Data shape: (233234, 597)
0.5544078066755165
PearsonRResult(statistic=0.4557654144520191, pvalue=0.0)

PearsonRResult(statistic=0.4431453643679412, pvalue=0.0)
PearsonRResult(statistic=0.7670512239998397, pvalue=0.0)


/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GroupKFold

LINEAR_FEATURE_NAMES = ['AdvantageP1']

gkf = GroupKFold(n_splits=10)
linear_model_rmses = []
for train_index, test_index in gkf.split(evaluated_games_df, groups=evaluated_games_df['GameRulesetName']):
    train_x = evaluated_games_df.iloc[train_index][LINEAR_FEATURE_NAMES]
    train_y = evaluated_games_df.iloc[train_index]['utility_agent1']

    test_x = evaluated_games_df.iloc[test_index][LINEAR_FEATURE_NAMES]
    test_y = evaluated_games_df.iloc[test_index]['utility_agent1']

    lr = LinearRegression()
    lr.fit(train_x, train_y)

    predictions = lr.predict(test_x)
    linear_model_rmses.append(mean_squared_error(test_y, predictions, squared=False))

print('10-fold linear RMSE:', np.mean(linear_model_rmses))

lr = LinearRegression()
lr.fit(organizer_df[LINEAR_FEATURE_NAMES], organizer_df['utility_agent1'])

print(lr.coef_, lr.intercept_)

predictions = lr.predict(organizer_df[LINEAR_FEATURE_NAMES])

print(mean_squared_error(organizer_df['utility_agent1'], predictions, squared=False))
print(pearsonr(organizer_df['utility_agent1'], predictions))

/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in ve

10-fold linear RMSE: 0.5646970217152508
[1.4765844] -0.7108871
0.5583498
PearsonRResult(statistic=0.443145363995553, pvalue=0.0)


In [22]:
print(np.mean(predictions), np.mean(organizer_df['utility_agent1']))
print(np.std(predictions), np.std(organizer_df['utility_agent1']))

dummy_prediction = np.mean(organizer_df['utility_agent1'])
dummy_predictions = [dummy_prediction] * len(organizer_df)

rmse = mean_squared_error(organizer_df['utility_agent1'], dummy_predictions, squared=False)

print(f'Dummy prediction: {dummy_prediction:.4f}')
print(f'Dummy RMSE: {rmse:.4f}')

0.044397697 0.04439766
0.2760111 0.62284565
Dummy prediction: 0.0444
Dummy RMSE: 0.6228


/home/jday/miniconda3/envs/torch2.3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
